In [1]:
!pip install pymysql
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb
import numpy as np

# Ask user for database username and password
user = input("Enter your username: ")
password = input("Enter your password: ")

# Connect to the database
connection = MySQLdb.connect(host='localhost', user='root', password='')

# Use cursor to execute SQL queries for the database
cursor = connection.cursor()

# Cleaning the sql file to be executed
def parse_sql(filename):
    data = open(filename, 'r').readlines()
    statements = []
    DELIMITER = ';'
    statement = ''

    for lineno, line in enumerate(data):
        if not line.strip():
            continue

        if line.startswith('--'):
            continue

        if 'DELIMITER' in line:
            DELIMITER = line.split()[1]
            continue

        if (DELIMITER not in line):
            statement += line.replace(DELIMITER, ';')
            continue

        if statement:
            statement += line
            statements.append(statement.strip())
            statement = ''
        else:
            statements.append(line.strip())
    return statements


# Checking if marshexplorer database already exists
cursor.execute("SHOW DATABASES LIKE 'marshexplorer'")
res = cursor.fetchone()

# Execute sql file if it exists
if res is None:
    # Reading sql statements from sql file and executing
    statements = parse_sql('marshexplorer.sql')
    for statement in statements:
        cursor.execute(statement)
    
    connection.commit()
else:
    cursor.execute("USE marshexplorer")


Enter your username: root
Enter your password: 


In [2]:
import pandas as pd

classifications_df = pd.read_csv('marsh-explorer-classifications.csv')
classifications_df

,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
0,384120667,MarshExplorer,2409282.0,dce34914fd75edc6d459,20498,Snails and Crabs,7.60,2021-12-17 20:48:51 UTC,NaN,NaN,"{""source"":""api"",""session"":""627a11675a1d3edfa03...","[{""task"":""T0"",""task_label"":""Please draw a box ...","{""71299458"":{""retired"":null,""Filename"":""thumbn...",71299458
1,384121150,MarshExplorer,2409282.0,dce34914fd75edc6d459,20498,Snails and Crabs,7.60,2021-12-17 20:50:32 UTC,NaN,NaN,"{""source"":""api"",""session"":""627a11675a1d3edfa03...","[{""task"":""T0"",""task_label"":""Please draw a box ...","{""71299460"":{""retired"":null,""Filename"":""thumbn...",71299460
2,384121246,MarshExplorer,2409282.0,dce34914fd75edc6d459,20498,Snails and Crabs,7.60,2021-12-17 20:50:54 UTC,NaN,NaN,"{""source"":""api"",""session"":""627a11675a1d3edfa03...","[{""task"":""T0"",""task_label"":""Please draw a box ...","{""71299459"":{""retired"":null,""Filename"":""thumbn...",71299459
3,386553971,MarshExplorer,2409282.0,f8f1eb7f2e5539f2b652,20498,Snails and Crabs,7.60,2021-12-31 19:18:21 UTC,NaN,NaN,"{""source"":""api"",""session"":""e4bce55a14978833bfe...","[{""task"":""T0"",""task_label"":""Please draw a box ...","{""71299459"":{""retired"":null,""Filename"":""thumbn...",71299459
4,386556946,MarshExplorer,2409282.0,1f7a8d34df5244542aff,20498,Snails and Crabs,7.60,2021-12-31 19:43:16 UTC,NaN,NaN,"{""source"":""api"",""session"":""e4bce55a14978833bfe...","[{""task"":""T0"",""task_label"":""Please draw a box ...","{""71299458"":{""retired"":null,""Filename"":""thumbn...",71299458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,422753967,AKantyka,2410780.0,882682384a116191c124,20498,Snails and Crabs,15.12,2022-06-22 21:03:12 UTC,NaN,NaN,"{""source"":""api"",""session"":""db4d65468487ad58021...","[{""task"":""T0"",""task_label"":""Please draw a box ...","{""75998687"":{""retired"":{""id"":102582890,""workfl...",75998687
2467,422753971,AKantyka,2410780.0,882682384a116191c124,20498,Snails and Crabs,15.12,2022-06-22 21:03:15 UTC,NaN,NaN,"{""source"":""api"",""session"":""db4d65468487ad58021...","[{""task"":""T0"",""task_label"":""Please draw a box ...","{""75998677"":{""retired"":{""id"":102582690,""workfl...",75998677
2468,422754009,AKantyka,2410780.0,882682384a116191c124,20498,Snails and Crabs,15.12,2022-06-22 21:03:34 UTC,NaN,NaN,"{""source"":""api"",""session"":""db4d65468487ad58021...","[{""task"":""T0"",""task_label"":""Please draw a box ...","{""75998680"":{""retired"":{""id"":102582718,""workfl...",75998680
2469,422754022,AKantyka,2410780.0,882682384a116191c124,20498,Snails and Crabs,15.12,2022-06-22 21:03:39 UTC,NaN,NaN,"{""source"":""api"",""session"":""db4d65468487ad58021...","[{""task"":""T0"",""task_label"":""Please draw a box ...","{""75998672"":{""retired"":{""id"":102582698,""workfl...",75998672


In [3]:
# Setting not logged in users id and username
classifications_df['user_id'] = classifications_df['user_id'].replace(np.nan, 0)
classifications_df.loc[classifications_df['user_name'].str.contains('not-logged-in'), 'user_name'] = 'UnloggedUser'

In [4]:
# Inserting users into user table
users_sql = "INSERT INTO user (user_id, username, expert) VALUES (%s, %s, %s)"

# Setting initial expert status to false (0)
expert = 0

# loop through the dataframe that has the csv file
for row in classifications_df.itertuples():
    
    # Getting all user ids from database
    cursor.execute("SELECT user_id FROM user where user_id=%s", (row.user_id))
    user_data = cursor.fetchall()
    
    # Checking if the user is already in database
    if not user_data:
        # If the expert value is true from csv data
        if row.expert == "TRUE":
            # Set expert status to true (1)
            expert = 1
        
        # Insert the data into the database
        cursor.execute(users_sql, (row.user_id, row.user_name, expert))
    else:
        continue

connection.commit()
    
    

In [5]:
# Inserting workflows into workflow table
workflows_sql = "INSERT INTO workflow (workflow_id, workflow_name) VALUES (%s, %s)"

# loop through the dataframe that has the csv file
for row in classifications_df.itertuples():
    
    # Getting all workflow ids from database
    cursor.execute("SELECT workflow_id FROM workflow where workflow_id=%s", (row.workflow_id))
    workflow_data = cursor.fetchall()
    
    # Checking if the workflow is already in database
    if not workflow_data:
        # Insert the data into the database
        cursor.execute(workflows_sql, (row.workflow_id, row.workflow_name))
    else:
        continue

connection.commit()

In [6]:
# Importing json to get the subject image_path from subject_data in csv
import json

# Inserting subject into subject table
subjects_sql = "INSERT INTO subject (subject_id, image_path, height, width) VALUES (%s, %s, %s, %s)"
image_path = ""
height = 0
width = 0

# loop through the dataframe that has the csv file
for row in classifications_df.itertuples():
    # Getting all subject ids from database
    cursor.execute("SELECT subject_id FROM subject where subject_id=%s", row.subject_ids)
    subject_data = cursor.fetchall()
    
    # Checking if the subject is already in database
    if not subject_data:
        # Loading in json and parsing it down to filename
        load_subject_data = json.loads(row.subject_data)
        subject_id_data = load_subject_data[str(row.subject_ids)]
        image_path = subject_id_data['Filename']
        
        # Insert the data into the database
        cursor.execute(subjects_sql, (row.subject_ids, image_path, height, width))
    else:
        continue
    
connection.commit()

In [7]:
# Inserting classifications into classification table
classifications_sql = "INSERT INTO classification (classification_id, workflow_id, user_id, date, gold_standard, workflow_version) VALUES (%s, %s, %s, %s, %s, %s)"

# Setting initial gold standard status to false (0)
gold_standard = 0

# Inserting annotations into annotations, circles and rectangles table
annotations_sql = "INSERT INTO annotation (annotation_id, subject_id, classification_id) VALUES (%s, %s, %s)"
circles_sql = "INSERT INTO circle (circle_id, annotation_id, tool_label, radius, x, y, angle) VALUES (null, %s, %s, %s, %s, %s, %s)"
rectangles_sql = "INSERT INTO rectangle (rectangle_id, annotation_id, tool_label, x, y, width, height) VALUES (null, %s, %s, %s, %s, %s, %s)"

# Initializing annotation_id
annotation_id = 0

# Alter Table SQL is only needed if auto_increment needs to be reset to 1 in database
cursor.execute("ALTER TABLE annotation AUTO_INCREMENT = 1")
cursor.execute("ALTER TABLE circle AUTO_INCREMENT = 1")
cursor.execute("ALTER TABLE rectangle AUTO_INCREMENT = 1")

# Get the max annotation id from annotation table
cursor.execute("SELECT max(annotation_id) from annotation")
max_annotation_id = cursor.fetchone()

# Start the annotation id at 1 if there is nothing is database
if max_annotation_id[0] is None:
    annotation_id = 1
else:
    # Add one to the current max annotation id
    annotation_id = max_annotation_id[0] + 1

# Open and load json file for tool labels
marsh_explorer_json = open('marsh_explorer.json')
marsh_explorer_data = json.load(marsh_explorer_json)

# loop through the dataframe that has the csv file
for row in classifications_df.itertuples():
    
    # Getting all classification ids from database
    cursor.execute("SELECT classification_id FROM classification where classification_id=%s", (row.classification_id))
    classification_data = cursor.fetchall()
    
    
    # Checking if the classification is already in database
    if not classification_data:
        
        # If the gold standard status is true from csv data
        if row.gold_standard == "TRUE":
            # Set gold standard status to true (1)
            gold_standard = 1

        # Insert the classification data into database
        cursor.execute(classifications_sql, (row.classification_id, row.workflow_id, row.user_id, row.created_at, gold_standard, row.workflow_version))
        
        # Loading in json for annotation data 
        load_annotation_data = json.loads(row.annotations)
    
        # Disable foreign key constraint to allow insertion of data for annotations
        cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
    
        # Insert annotation data into the annotations table from csv file
        cursor.execute(annotations_sql, (annotation_id, row.subject_ids, row.classification_id))
        
        # Loop through the data from the loaded json data that was from the csv file
        for data in load_annotation_data:
            # Get the values of value from the data
            for value in data['value']:
                
                # Loop through the marsh explorer json to get tool labels
                for tool_label in marsh_explorer_data['tool_labels']:
                    
                    # Rectangles Table
                    # Loop to get all the names of subject of interests that uses rectangle annotation
                    for name in tool_label['rectangle']:
                        # If the tool label is equal to the json names of interest then insert rectangle data into rectangles table
                        if value['tool_label'] == str(name['name']):
                            cursor.execute(rectangles_sql, (annotation_id, value['tool_label'], value['x'], value['y'], value['width'], value['height']))
                    
                    # Circles Table
                    # Loop to get all the names of subject of interests that uses circle annotation
                    for name in tool_label['circle']:
                        # If the tool label is equal to the json names of interest then insert circle data into circles table
                        if value['tool_label'] == str(name['name']):
                            cursor.execute(circles_sql, (annotation_id, value['tool_label'], value['r'], value['x'], value['y'], value['angle']))
        
        # Increment annotation id
        annotation_id = annotation_id + 1
        
    else:
        continue
        
connection.commit()

In [8]:
subjects_df = pd.read_csv('marsh-explorer-subjects.csv')
subjects_df

,subject_id,project_id,workflow_id,subject_set_id,metadata,locations,classifications_count,retired_at,retirement_reason,created_at,updated_at
0,458021,17733,NaN,101038,"{""link"":""https://www.flickr.com/photos/aigle_d...","{""0"":""https://panoptes-uploads.zooniverse.org/...",0,NaN,NaN,2015-05-25 21:02:58 UTC,2015-05-25 21:02:58 UTC
1,458022,17733,NaN,101038,"{""link"":""https://www.flickr.com/photos/aigle_d...","{""0"":""https://panoptes-uploads.zooniverse.org/...",0,NaN,NaN,2015-05-25 21:02:59 UTC,2015-05-25 21:02:59 UTC
2,458023,17733,NaN,101038,"{""link"":""https://www.flickr.com/photos/hellie5...","{""0"":""https://panoptes-uploads.zooniverse.org/...",0,NaN,NaN,2015-05-25 21:03:01 UTC,2015-05-25 21:03:01 UTC
3,458024,17733,NaN,101038,"{""link"":""https://www.flickr.com/photos/odoketa...","{""0"":""https://panoptes-uploads.zooniverse.org/...",0,NaN,NaN,2015-05-25 21:03:02 UTC,2015-05-25 21:03:02 UTC
4,458025,17733,NaN,101038,"{""link"":""https://www.flickr.com/photos/bevgood...","{""0"":""https://panoptes-uploads.zooniverse.org/...",0,NaN,NaN,2015-05-25 21:03:03 UTC,2015-05-25 21:03:03 UTC
...,...,...,...,...,...,...,...,...,...,...,...
4158,75998727,17733,20498.0,104685,"{""Filename"":""imgs_1_6_21.jpg""}","{""0"":""https://panoptes-uploads.zooniverse.org/...",22,2022-06-14 19:43:29 UTC,classification_count,2022-05-16 16:08:57 UTC,2022-05-16 16:08:57 UTC
4159,75998728,17733,20498.0,104685,"{""Filename"":""imgs_1_6_22.jpg""}","{""0"":""https://panoptes-uploads.zooniverse.org/...",25,2022-06-15 00:24:21 UTC,classification_count,2022-05-16 16:08:58 UTC,2022-05-16 16:08:58 UTC
4160,76836327,17733,NaN,105304,"{""Filename"":""thumbnail_Row36_14_7631to7790_DSC...","{""0"":""https://panoptes-uploads.zooniverse.org/...",0,NaN,NaN,2022-06-01 17:12:51 UTC,2022-06-01 17:12:51 UTC
4161,76836328,17733,NaN,105304,"{""Filename"":""thumbnail_run_15_Imaging_8425_5.j...","{""0"":""https://panoptes-uploads.zooniverse.org/...",0,NaN,NaN,2022-06-01 17:12:51 UTC,2022-06-01 17:12:51 UTC


In [9]:
# Updating subject set location information
subject_set_sql = "UPDATE subject SET subject_set_id=%s, subject_set_name=%s, location=%s WHERE subject_id=%s"

# Open and load json file for subject sets
marsh_explorer_json = open('marsh_explorer.json')
marsh_explorer_data = json.load(marsh_explorer_json)

subject_set_name = ""
location = ""

# loop through the dataframe that has the csv file
for row in subjects_df.itertuples():
    
    # Getting all subject ids from database
    cursor.execute("SELECT subject_id FROM subject where subject_id=%s", (row.subject_id))
    subject_data = cursor.fetchall()
    
    # Setting subject set name and location based on the subject set id
    if subject_data:
        
        # Loop through marsh explorer json to get the subject sets
        for subject_set in marsh_explorer_data['subject_sets']:
            # If the subject set id from csv is equal to the json data subject id, set location and subject set name accordingly
            if row.subject_set_id == int(subject_set['subject_set_id']):
                subject_set_name = subject_set['subject_set_name']
                location = subject_set['location']
                
        cursor.execute(subject_set_sql, (row.subject_set_id, subject_set_name, location, row.subject_id))
        
    else:
        continue
    
connection.commit()